# Application of Bootstrap samples in Random Forest

In [1]:
import numpy as np # importing numpy for numerical computation
# here we are using sklearn's boston dataset
from sklearn.datasets import fetch_openml
boston = fetch_openml(name='boston')
from sklearn.metrics import mean_squared_error # importing mean_squared_error metric
import warnings
warnings.filterwarnings("ignore")

C:\Users\svani\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\datasets\_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
C:\Users\svani\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
x=boston.data #independent variables
y=boston.target #target variable

## <font color='red'><b>Task 1</b></font>

<font color='red'> <b>Step - 1</b></font>

*  <font color='blue'><b>Creating samples</b></font><br>
    <b> Randomly create 30 samples from the whole boston data points</b>
    *  Creating each sample: Consider any random 303(60% of 506) data points from whole data set and then replicate any 203 points from the sampled points
    
     For better understanding of this procedure lets check this examples, assume we have 10 data points [1,2,3,4,5,6,7,8,9,10], first we take 6 data points randomly , consider we have selected [4, 5, 7, 8, 9, 3] now we will replicate 4 points from [4, 5, 7, 8, 9, 3], consder they are [5, 8, 3,7] so our final sample will be [4, 5, 7, 8, 9, 3, 5, 8, 3,7]
* <font color='blue'><b> Create 30 samples </b></font>
    *  Note that as a part of the Bagging when you are taking the random samples <b>make sure each of the sample will have different set of columns</b><br>
Ex: Assume we have 10 columns[1 ,2 ,3 ,4 ,5 ,6 ,7 ,8 ,9 ,10] for the first sample we will select [3, 4, 5, 9, 1, 2] and for the second sample  [7, 9, 1, 4, 5, 6, 2] and so on...
Make sure each sample will have atleast 3 feautres/columns/attributes

* <font color='red'><b> Note - While selecting the random 60% datapoints from the whole data, make sure that the selected datapoints are all exclusive, repetition is not allowed. </b></font>

In [3]:
x.shape

(506, 13)

In [4]:
import numpy as np
import pandas as pd
import random 


def generating_samples(input_data, target_data): 
    
    np.random.seed(123) # Set random seed

    # random.choice to generate random indices without replacement
    # Selecting_rows => Getting 303 random row indices from the input_data, without replacement
    Selecting_rows = input_data.sample(n=303, replace=False).index

    # now we will replicate 203 points from above selected rows
    # Replacing Rows => Extracting 206 random row indices from the Selecting_rows
    Replacing_rows = pd.Series(Selecting_rows).sample(n=203, replace=False).values

    # Selecting_columns => Getting 3 to 13 random column indices from input_data
    Selecting_columns = random.randint(3, 13)
    columns_selected = np.array(random.sample(range(0, 13), Selecting_columns ))

    # sample_data => input_data.iloc[Selecting_rows[:,None], Selecting_columns]
    sample_data = input_data.iloc[Selecting_rows, columns_selected]
    
    # target_of_sample_data => target_data[Selecting_rows]
    target_of_sample_data = target_data[Selecting_rows]
    
    # Replicating data
    # Now Replication of Data for 203 data points out of 303 selected points
    replicated_sample_data = input_data.iloc[Replacing_rows, columns_selected ]
    target_of_replicated_sample_data = target_data[Replacing_rows]
    
    # Concatenating data
    final_sample_data = np.vstack((sample_data.values, replicated_sample_data.values ))
    final_target_data = np.vstack((target_of_sample_data.to_numpy().reshape(-1, 1), target_of_replicated_sample_data.to_numpy().reshape(-1, 1)))
            
    return final_sample_data, final_target_data, Selecting_rows, columns_selected

a,b,c,d = generating_samples(x, y)

<font color='cyan'> <b> Grader function - 1 </b> </fongt>

In [8]:
def grader_samples(a,b,c,d):
    length = (len(a)==506  and len(b)==506)
    sampled = (len(a)-len(set([str(i) for i in a]))==203)
    rows_length = (len(c)==303)
    column_length= (len(d)>=3)
    assert(length and sampled and rows_length and column_length)
    return True
a,b,c,d = generating_samples(x, y)
grader_samples(a,b,c,d)

True

*  <font color='blue'> <b>Create 30 samples </b>

In [9]:
# Create empty lists to store the samples
list_input_data = []
list_output_data = []
list_selected_row = []
list_selected_columns = []

# Loop to generate 30 samples
for i in range(30):
    # Generate a sample
    a, b, c, d = generating_samples(x, y)
    
    # Append the sample to the respective lists
    list_input_data.append(a)
    list_output_data.append(b)
    list_selected_row.append(c)
    list_selected_columns.append(d)


<font color='cyan'> <b>Grader function - 2 </b></font>

In [10]:
def grader_30(a):
    assert(len(a)==30 and len(a[0])==506)
    return True
grader_30(list_input_data)

True

<font color='red'><b>Step - 2 </b></font>

<font color='blue'><b>Building High Variance Models on each of the sample and finding train MSE value</b></font>

*  Build a regression trees on each of 30 samples.
*  Computed the predicted values of each data point(506 data points) in your corpus.
*  Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{30}\sum_{k=1}^{30}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$
*  Now calculate the $MSE =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$

In [11]:
from sklearn.tree import DecisionTreeRegressor

# Create an empty list to store the trained models
tree_models = []

# Train a decision tree model for each of the 30 samples
for i in range(30):
    tree_model = DecisionTreeRegressor()
    tree_model.fit(list_input_data[i], list_output_data[i])
    
    # Append the trained model to the list
    tree_models.append(tree_model)

In [12]:
# Make predictions for each model and store the predicted values in a list
list_predicted_values = []
for i in range(len(tree_models)):
    # Predict the output values for the i-th sample using the corresponding model
    predicted_values = tree_models[i].predict(list_input_data[i])
    list_predicted_values.append(predicted_values)  

After getting predicted_y for each data point, we can use sklearns mean_squared_error to calculate the MSE between predicted_y and actual_y.

In [13]:
# Calculate the mean squared error for each sample's predictions and store them in a list
from sklearn.metrics import mean_squared_error

mse_values = []
for i in range(len(list_predicted_values)):
    # Calculate the mean squared error between the i-th sample's actual output and predicted output
    mse = mean_squared_error(list_output_data[i], list_predicted_values[i])
    mse_values.append(mse)
    
print(mse_values)

[0.0, 0.0, 0.0, 0.0, 7.483265504072839e-32, 7.483265504072839e-32, 7.483265504072839e-32, 0.0, 0.0, 7.483265504072839e-32, 0.0, 0.16527009222661393, 7.483265504072839e-32, 0.0, 1.9410724637681158, 7.483265504072839e-32, 7.483265504072839e-32, 0.0, 0.0, 0.0, 7.483265504072839e-32, 0.0, 0.06449440052700924, 0.0, 0.0, 0.19963109354413697, 0.16527009222661393, 0.0, 7.483265504072839e-32, 0.0]


<font color='blue'><b>Step - 3 </b></font>

Now calculate the $OOB Score =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.

In [14]:
# Create an empty list to store the indices of samples that do not include a data point
oob_indices = []

for i in range(506):
    # Identify the indices of all samples that do not include the i-th data point
    sample_indices = []
    for j in range(len(list_input_data)):
        if i not in list_input_data[j]:
            sample_indices.append(j)
    # Append these indices to the list of OOB indices
    oob_indices.append(sample_indices)


In [16]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Create an empty list to store the OOB scores
oob_scores = []

for i in range(506):
    # Identify the indices of all samples that do not include the i-th data point
    sample_indices = oob_indices[i]
    # Use these samples as the training data to build the regression tree model
    X_train = a[sample_indices]
    y_train = b[sample_indices]
    model = DecisionTreeRegressor(random_state=42)
    model.fit(X_train, y_train)
    # Use the data points that were not included in the subset as the OOB data
    oob_data = a[i].reshape(1, -1)
    # Compute the predicted value of the target variable for the OOB data
    y_pred = model.predict(oob_data)
    # Compute the OOB score for the i-th data point
    y_actual = b[i]
    oob_score = (y_pred - y_actual) ** 2
    # Append the OOB score to the list of OOB scores
    oob_scores.append(oob_score)

# Compute the overall OOB score by averaging the OOB scores for all 506 data points
mean_oob_score = np.mean(oob_scores)
print("Overall OOB score:", mean_oob_score)


Overall OOB score: 67.0459486166008


# <font color='red'><b>Task 2</b></font>

*  <font color='blue'><b>Computing CI of OOB Score and Train MSE</b></font>
  *   Repeat Task 1 for 35 times, and for each iteration store the Train MSE and OOB score </li>
<li> After this we will have 35 Train MSE values and 35 OOB scores </li>
<li> using these 35 values (assume like a sample) find the confidence intravels of MSE and OOB Score </li>
<li> you need to report CI of MSE and CI of OOB Score </li>
<li> Note: Refer the Central_Limit_theorem.ipynb to check how to find the confidence intravel</li>
</ol>